## ReMKiT1D Analysis 

This notebook provides examples of most often used ReMKiT1D data analysis/plotting scripts

In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn 
import sys
sys.path.append('../')
from RMK_support import Grid,VariableContainer
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

### HDF5 Path

In [ ]:
hdf5Filepath = "./" #Default value

### Grid data

Currently best to copy this from your initialization script (also see pkl_analysis for when the original pickled RKWrapper object is available)

In [ ]:
xGrid = np.linspace(0.5,15,128) 
vGrid = np.logspace(-2,0,80) 
lMax = 3
gridObj = Grid(xGrid,vGrid,lMax,interpretVGridAsWidths=True,isLengthInMeters=True)
gridObj.xJacobian = np.ones(129) 

### Example data

Create some example data for plotting

In [ ]:
n = np.ones(gridObj.numX())
T = np.linspace(10.0,5.0,gridObj.numX())
f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
# Set harmonic with l = 0 to a maxwellian using standard normalizaion (see above)
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = np.pi**(-1.5) * n[i] * np.exp(-gridObj.vGrid**2/T[i])

vc = VariableContainer(gridObj)

vc.setVariable("n",n)
vc.setVariable("T",T,units="eV")
vc.setVariable("f",f,isDistribution=True)
vc.setVariable("time",isScalar=True) 

exampleFilenames = [hdf5Filepath+f'Example{i}.h5' for i in range(6)]

for i in range(6):
    buffer = (i+1)*np.ones(gridObj.numX())
    vc.dataset["n"][:] = buffer
    buffer = vc.dataset["T"].copy()
    buffer.data =  np.linspace(10.0 - i,5.0+i,gridObj.numX())
    vc.dataset["T"][:] = buffer
    buffer = (i+1)*np.ones(1)*0.1
    vc.dataset["time"][:] = buffer

    io.writeRMKHDF5(vc,filepath=exampleFilenames[i]) 


### Prepare variable container for loading data

In [ ]:
vcLoad = VariableContainer(gridObj)
vcLoad.setVariable("n",units='$10^{19} m^{-3}$')
vcLoad.setVariable("T",units="eV")
vcLoad.setVariable("f",isDistribution=True)
vcLoad.setVariable("time",isScalar=True) 

### Load data from example files

In [ ]:
loadedData = io.loadFromHDF5(vcLoad,filepaths=exampleFilenames)
loadedData

### Set some names and units for the coordinates

In [ ]:
loadedData.coords['x'].attrs['units'] = "m"
loadedData.coords['v'].attrs['units'] = "$v_{th}$"
loadedData.coords['time'].attrs['standard_name'] = 't'
loadedData.coords['time'].attrs['units'] = "s"


### Quick plotting using inbuilt xarray matplotlib support

See [xarray plotting guide](https://xarray.pydata.org/en/stable/user-guide/plotting.html#)

In [ ]:
plot = loadedData['T'][{"time":0}].plot()
plt.title('Temperature')

In [ ]:
plot = loadedData['T'][{"x":100}].plot()
plt.title('Temperature at x=0')

In [ ]:
plot = loadedData['f'][{"x":0,"h":0,"time":0}].plot(yscale='log')
plt.title('$f_0$ at x = 0')
plt.ylabel("")
plt.xlabel("E (eV)")
plt.xscale('function', functions=(lambda x : x**2,lambda x : x**(1/2)))
xticks = np.linspace(min(loadedData['v'].data**2),max(loadedData['v'].data**2),10)
plt.xticks(xticks**(1/2),map(str,np.around(xticks,1)))
plt.show()

### Explore single fluid variable at different points in time using HoloMap

See [HoloViews](https://holoviews.org/index.html)

In [ ]:
dataName = 'T'

curveDict = {t: hv.Curve(loadedData[dataName][{"time":t}]) for t in range(6)}
kdims = [hv.Dimension(('time', 'Time'),unit=loadedData.coords["time"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims)

### Compare data using HoloViews overlay and layout features

In [ ]:
# hook used for matplotlib axes formatting, move this into separate module
def format_axes(plot, element):
    ax = plot.handles['axis']
    ax.get_xaxis().set_major_locator(mpl.ticker.AutoLocator())
    ax.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax.get_yaxis().set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    ax.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())

dataName1 = 'T'
dataName2 = 'n'
plotName1 = 'Temperature'
plotName2 = 'Density'

# Fontsize dictionary useful to keep around
fontsize={
    'title': 15, 
    'labels': 14, 
    'xticks': 10, 
    'yticks': 10,
}

timesToCompare = [0,2,3]

curves1 = [hv.Curve(loadedData[dataName1][{"time":t}]) for t in timesToCompare]
overlay1 = hv.Overlay(curves1)
curves2 = [hv.Curve(loadedData[dataName2][{"time":t}],label=f't = {loadedData.coords["time"].data[t]:.2f} '+str(loadedData.coords["time"].attrs["units"])) for t in timesToCompare]
overlay2 = hv.Overlay(curves2).opts(legend_position='right')

layout = overlay1.relabel(plotName1) + overlay2.relabel(plotName2)

layout.opts(opts.Curve(linewidth=2.0,color=hv.Palette('copper'),fontsize=fontsize,hooks=[format_axes]))
layout.opts(sublabel_format="{alpha}",sublabel_size=15)



### Explore multiple variables interactively using basic dashboard

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,gridObj)

dashboard.fluid2Comparison()
